In [ ]:
#| default_exp tools

# LLM tools
> Helpful tools for running cli commands and reading, modifying, and creating files in python. This is used primarily for AI's in tool loops for automating tasks involving the filesystem.

In [ ]:
#| export
from fastcore.utils import *
from shlex import split
from subprocess import run, DEVNULL

In [ ]:
#| hide
from fastcore.test import test_eq
from toolslm.funccall import get_schema
import inspect

## Bash Tools

In [ ]:
#| export
def run_cmd(
    cmd:str, # The command name to run
    argstr:str='', # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run `cmd` passing split `argstr`, optionally checking for allowed argstr"
    if disallow_re and re.search(disallow_re, argstr): return 'Error: args disallowed'
    if allow_re    and re.search(   allow_re, argstr): return 'Error: args not allowed'
    try: outp = run([cmd] + split(argstr), text=True, stdin=DEVNULL, capture_output=True)
    except Exception as e: return f'Error running cmd: {str(e)}'
    res = outp.stdout
    if res and outp.stderr: res += '\n'
    return res + outp.stderr

With this little function, we can now run any cli command:

In [ ]:
print(run_cmd('ls')[:128])

000_tour.ipynb
00_test.ipynb
01_basics.ipynb
02_foundation.ipynb
03_xtras.ipynb
03a_parallel.ipynb
03b_net.ipynb
04_docments.ipy


Note that, for tool safety, this is not passed through the shell, so wildcards, env vars, etc will not work:

In [ ]:
print(run_cmd('ls', 'f*')[:128])

ls: f*: No such file or directory



Let's create some useful functions from this that will allow for searching, reading and modifing content on the file system.

In [ ]:
#| export
def rg(
    argstr:str, # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run the `rg` command with the args in `argstr` (no need to backslash escape)"
    return run_cmd('rg', '-n '+argstr, disallow_re=disallow_re, allow_re=allow_re)

In [ ]:
rg('fast.ai CNAME')

'1:fastcore.fast.ai\n'

Functions implemented with `run_cmd` like this one can be passed regexps to allow or disallow arg strs, i.e to block parent or root directories:

In [ ]:
disallowed = r' /|\.\.'
rg('info@fast.ai ..', disallow_re=disallowed)

'Error: args disallowed'

In [ ]:
rg('info@fast.ai /', disallow_re=disallowed)

'Error: args disallowed'

In [ ]:
print(rg('fast.ai CNAME', disallow_re=disallowed))

1:fastcore.fast.ai



NB: These tools have special behavior around errors. Since these have been speficially designed for work with LLMs, any exceptions created from there use is returned as a string to help them debug their work.

In [ ]:
run_cmd('asdfe')

"Error running cmd: [Errno 2] No such file or directory: 'asdfe'"

In [ ]:
#| export
def sed(
    argstr:str, # All args to the command, will be split with shlex
    disallow_re:str=None, # optional regex which, if matched on argstr, will disallow the command
    allow_re:str=None # optional regex which, if not matched on argstr, will disallow the command
):
    "Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)"
    return run_cmd('sed', argstr, allow_re=allow_re, disallow_re=disallow_re)

In [ ]:
print(sed('-n "1,5 p" _quarto.yml'))

project:
  type: website
  pre-render: 
    - pysym2md --output_file apilist.txt fastcore
  post-render: 



In [ ]:
# Print line numbers too
print(sed('-n "1,5 {=;p;}" _quarto.yml'))

1
project:
2
  type: website
3
  pre-render: 
4
    - pysym2md --output_file apilist.txt fastcore
5
  post-render: 



## Text Edit Tools

Python implementations of the text editor tools from [Anthropic](https://docs.claude.com/en/docs/agents-and-tools/tool-use/text-editor-tool). These tools are especially useful in an AI's tool loop. See [`claudette`](https://claudette.answer.ai/text_editor.html) for examples.

In [ ]:
#| export
def view(
    path:str, # Path to directory or file to view
    view_range:tuple[int,int]=None, # Optional 1-indexed (start, end) line range for files, end=-1 for EOF
    nums:bool=False # Whether to show line numbers
):
    'View directory or file contents with optional line range and numbers'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        header = None
        if p.is_dir():
            files = [str(f) for f in p.glob('**/*') 
                    if not any(part.startswith('.') for part in f.relative_to(p).parts)]
            lines = files
            header = f'Directory contents of {p}:'
        else: lines = p.read_text().splitlines()
        s, e = 1, len(lines)
        if view_range:
            s,e = view_range
            if not (1<=s<=len(lines)): return f'Error: Invalid start line {s}'
            if e!=-1 and not (s<=e<= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
        if nums: lines = [f'{i+s:6d} │ {l}' for i, l in enumerate(lines)]
        content = '\n'.join(lines)
        return f'{header}\n{content}' if header else content
    except Exception as e: return f'Error viewing: {str(e)}'

You can specify line ranges and whether to have the output contain line numbers:

In [ ]:
print(view('_quarto.yml', (1,10), nums=True))

     1 │ project:
     2 │   type: website
     3 │   pre-render: 
     4 │     - pysym2md --output_file apilist.txt fastcore
     5 │   post-render: 
     6 │     - llms_txt2ctx llms.txt --optional true --save_nbdev_fname llms-ctx-full.txt
     7 │     - llms_txt2ctx llms.txt --save_nbdev_fname llms-ctx.txt
     8 │   resources: 
     9 │     - "*.txt"
    10 │   preview:


Here's what the output looks like when viewing a directory:

In [ ]:
print(view('.', (1,5)))

Directory contents of /Users/jhoward/aai-ws/fastcore/nbs:
/Users/jhoward/aai-ws/fastcore/nbs/llms.txt
/Users/jhoward/aai-ws/fastcore/nbs/000_tour.ipynb
/Users/jhoward/aai-ws/fastcore/nbs/parallel_test.py
/Users/jhoward/aai-ws/fastcore/nbs/_quarto.yml
/Users/jhoward/aai-ws/fastcore/nbs/08_style.ipynb


In [ ]:
#| export
def create(
    path: str, # Path where the new file should be created
    file_text: str, # Content to write to the file
    overwrite:bool=False # Whether to overwrite existing files
) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p}.'
    except Exception as e: return f'Error creating file: {str(e)}'

In [ ]:
print(create('test.txt', 'Hello, world!'))
f = Path('test.txt')
test_eq(f.exists(), True)
print('Contents:\n', view(f, nums=True))

Created file test.txt.
Contents:
      1 │ Hello, world!


In [ ]:
#| export
def insert(
    path: str, # Path to the file to modify
    insert_line: int, # Line number where to insert (0-based indexing)
    new_str: str # Text to insert at the specified line
) -> str:
    'Insert new_str at specified line number'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [ ]:
insert(f, 0, 'Let\'s add a new line')
print(view(f, nums=True))

     1 │ Let's add a new line
     2 │ Hello, world!


In [ ]:
#| export
def str_replace(
    path: str, # Path to the file to modify
    old_str: str, # Text to find and replace
    new_str: str # Text to replace with
) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
        content = p.read_text()
        count = content.count(old_str)
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [ ]:
str_replace(f, 'new line', '')
print(view(f, nums=True))

     1 │ Let's add a 
     2 │ Hello, world!


In [ ]:
#| export
def strs_replace(
    path:str, # Path to the file to modify
    old_strs:list[str], # List of strings to find and replace
    new_strs:list[str], # List of replacement strings (must match length of old_strs)
):
    "Replace for each str pair in old_strs,new_strs call `str_replace"
    res = [str_replace(path, old, new) for (old,new) in zip(old_strs,new_strs)]
    return 'Results for each replacement:\n' + '; '.join(res)

In [ ]:
strs_replace(f, ["add a new line", "world!"], ["just say", "friends!\nNice to see you."])
print(view(f, nums=True))

     1 │ Let's add a 
     2 │ Hello, friends!
     3 │ Nice to see you.


In [ ]:
#| export
def replace_lines(
    path:str, # Path to the file to modify
    start_line:int, # Starting line number to replace (1-based indexing)
    end_line:int, # Ending line number to replace (1-based indexing, inclusive)
    new_content:str, # New content to replace the specified lines
):
    "Replace lines in file using start and end line-numbers (index starting at 1)"
    if not (p := Path(path)).exists(): return f"Error: File not found: {p}"
    content = p.readlines()
    if not new_content.endswith('\n'): new_content+='\n'
    content[start_line-1:end_line] = [new_content]
    p.write_text(''.join(content))
    return f"Replaced lines {start_line} to {end_line}."

In [ ]:
replace_lines('test.txt', 1, 2, 'Replaced first two lines')
print(view('test.txt', nums=True))

     1 │ Replaced first two lines
     2 │ Nice to see you.


In [ ]:
f.unlink()

In [ ]:
#| hide
for f,o in list(globals().items()):
    if callable(o) and hasattr(o, '__module__') and o.__module__ == '__main__' and not f.startswith(('_', 'receive_')):
        assert f == get_schema(globals()[f])['name']

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev import nbdev_export
nbdev_export()